In [ ]:
import pandas as pd
from argparse import ArgumentParser

suspend_ts_ns = 200000
restart_ts_ns = 600000

routerfile = None
terminalfile = None

link_bw = 25

In [ ]:
# Set data files

#### dumbbell  jlse 24
#terminalfile = 'dumbbell/fixed-rate.jlse-equal/terminal-packet-stats-0-178945-1739156161' #equal-equal
#terminalfile = 'dumbbell/fixed-rate.jlse-fair/terminal-packet-stats-0-179119-1739157348' #fixed-fair
terminalfile = 'dumbbell/changing-rate.jlse/terminal-packet-stats-0-178358-1739140601' #changing


#### multiflow

#latest terminalfile = 'multiflow/changing-rates/mforiginal-term' #changing
#terminalfile = 'multiflow/changing-rates-jlse/terminal-packet-stats-0-153895-1738824802' #changing.jlseabs



In [ ]:

if routerfile:
    # Read Router Data
    rdf = pd.read_table(routerfile, sep=' ')

    # split credit column
    rdf[['vc0', 'vc1', 'vc2', 'vc3']] = rdf['downstream-credits'].str.extract('(\d+):(\d+):(\d+):(\d+)', expand=False)
    
    # Drop unused columns and sort
    cols_to_drop = [col for col in rdf.columns if col.startswith('qos-')]
    cols_to_drop.append('Unnamed: 0')
    cols_to_drop.append('busy-time')
    cols_to_drop.append('vc-occupancy')
    cols_to_drop.append('downstream-credits')
    rdf = rdf.drop(cols_to_drop, axis=1)

    rdf = rdf.sort_values(['time-stamp','router-id','port-id'])

    # Get stats at suspend
    df_suspend = rdf[  rdf['time-stamp'] == suspend_ts_ns ]
    df_suspend = rdf[  rdf['time-stamp'] == suspend_ts_ns ]

    print('\n')
    print('ROUTER STATS AT SUSPEND (time in ns)')
    print('====================================')
    print(df_suspend)

    # Get stats at restart
    rdf_restart = rdf[  rdf['time-stamp'] == restart_ts_ns ]
    rdf_restart = rdf[  rdf['time-stamp'] == restart_ts_ns ]

    print('\n')
    print('ROUTER STATS AT SUSPEND (time in ns)')
    print('====================================')
    print(rdf_restart)


if terminalfile:
    # Read Terminal Data
    tdf = pd.read_table(terminalfile, sep=' ')

    # split credit column
    tdf[['vc0', 'vc1', 'vc2', 'vc3']] = tdf['downstream-credits'].str.extract('(\d+):(\d+):(\d+):(\d+)', expand=False)
    
    # Drop unused columns and sort
    cols_to_drop = [col for col in tdf.columns if col.startswith('qos-')]
    cols_to_drop = [col for col in tdf.columns if col.startswith('vc')]
    cols_to_drop.append('Unnamed: 0')
    cols_to_drop.append('qos-level')
    cols_to_drop.append('downstream-credits')
    tdf = tdf.drop(cols_to_drop, axis=1)
    
    # Remove empty records with no bandwidth
    tdf = tdf[tdf['bw-consumed'] != 0]
    
    # Covert values
    tdf['time-stamp'] = tdf['time-stamp'].apply(lambda x: x / 1000) # Convert ns to us
    tdf['bw-consumed'] = tdf['bw-consumed'].apply(lambda x: (x * link_bw)/100) # Convert % to bw

    
    tdf = tdf.sort_values(['time-stamp','term-id'])
    print(tdf['term-id'].unique())
    #print('\n')
    #print('ALL TERMINAL STATS        (time in ns)')
    #print('======================================')
    #print(tdf[  tdf['term-id'] == 1 ])
    #print(tdf)
    '''
    # Get stats at suspend
    tdf_suspend = tdf[  tdf['time-stamp'] == suspend_ts_ns ]
    tdf_suspend = tdf[  tdf['time-stamp'] == suspend_ts_ns ]

    print('\n')
    print('TERMINAL STATS AT SUSPEND (time in ns)')
    print('======================================')
    print(tdf_suspend)

    # Get stats at restart
    tdf_restart = tdf[  tdf['time-stamp'] == restart_ts_ns ]
    tdf_restart = tdf[  tdf['time-stamp'] == restart_ts_ns ]

    print('\n')
    print('TERMINAL STATS AT RESTART (time in ns)')
    print('======================================')
    print(tdf_restart)
    '''


In [ ]:
# DUMBBELL2
import matplotlib.pyplot as plt
from matplotlib import style 

plt.style.use('default') 

x = tdf[tdf['term-id'] == 1]['time-stamp']
y0 = tdf[tdf['term-id'] == 0]['bw-consumed']
y1 = tdf[tdf['term-id'] == 1]['bw-consumed']
y2 = tdf[tdf['term-id'] == 2]['bw-consumed']


fig, axs = plt.subplots(1, 3)

axs[0].plot(x, y0, marker='.', fillstyle='none', color='y')
axs[0].axhline(y=1, color='k', linestyle='--', label = 'steady state') 
#axs[0].axhline(y=12.5, color='r', linestyle='--', label = '1st steady state') 
#axs[0].axhline(y=22.72, color='b', linestyle='--', label = '2nd steady state') 
axs[0].set_title("Flow 0", fontsize=10)

axs[1].plot(x, y1, marker='.', fillstyle='none', color='y')
axs[1].axhline(y=10, color='k', linestyle='--', label = 'steady state')
#axs[1].axhline(y=12.5, color='r', linestyle='--', label = '1st steady state')
#axs[1].axhline(y=2.27, color='b', linestyle='--', label = '2nd steady state') 
axs[1].set_title("Flow 1", fontsize=10)

axs[2].plot(x, y2, marker='.', fillstyle='none', color='y')
axs[2].axhline(y=14, color='k', linestyle='--', label = 'steady state')
#axs[2].axhline(y=12.5, color='r', linestyle='--', label = '1st steady state')
#axs[2].axhline(y=2.27, color='b', linestyle='--', label = '2nd steady state') 
axs[2].set_title("Flow 2", fontsize=10)


for ax in axs.flat:
    ax.set (xlabel='Time (us)', ylabel='Flow Injection Rate\n(GB/s)')
    #ax.set_xlim(0, 25)
    ax.set_ylim(0, 28)
    ax.legend()
    ax.label_outer()

fig.set_figheight(1.3)


# Save the entire figure
#fig.savefig("figs/single-congestion-changing-rates.png")

In [ ]:
# DUMBBELL1
import matplotlib.pyplot as plt
from matplotlib import style 

plt.style.use('default') 

x = tdf[tdf['term-id'] == 1]['time-stamp']
y0 = tdf[tdf['term-id'] == 0]['bw-consumed']
y1 = tdf[tdf['term-id'] == 1]['bw-consumed']


fig, axs = plt.subplots(1, 2)

axs[0].plot(x, y0, marker='.', fillstyle='none', color='y')
axs[0].axhline(y=12.5, color='k', linestyle='--', label = 'steady state') 
axs[0].set_title("Flow 0", fontsize=10)

axs[1].plot(x, y1,  marker='.', fillstyle='none', color='y')
axs[1].axhline(y=12.5, color='k', linestyle='--', label = 'steady state')
#axs[1].axhline(y=2.27, color='b', linestyle='--', label = '2nd steady state') 
axs[1].set_title("Flow 1", fontsize=10)


for ax in axs.flat:
    ax.set (xlabel='Time (us)', ylabel='Flow Injection Rate\n(GB/s)')
    #ax.set_xlim(0, 25)
    ax.set_ylim(0, 28)
    ax.legend()
    ax.label_outer()

fig.set_figheight(1.3)


# Save the entire figure
fig.savefig("figs/single-congestion-changing-rates.png")


In [ ]:
# DUMBBELL3

import matplotlib.pyplot as plt
from matplotlib import style 

plt.style.use('default') 

#tdf = tdf[tdf['time-stamp'] < 100]

x0 = tdf[tdf['term-id'] == 0]['time-stamp']
y0 = tdf[tdf['term-id'] == 0]['bw-consumed']

x1 = tdf[tdf['term-id'] == 1]['time-stamp']
y1 = tdf[tdf['term-id'] == 1]['bw-consumed']

x3 = tdf[tdf['term-id'] == 2]['time-stamp']
y3 = tdf[tdf['term-id'] == 2]['bw-consumed']


p1end = 7
p2end = 12


fig, axs = plt.subplots(1, 3)

axs[0].plot(x0, y0, marker='.', fillstyle='none', color='y')
axs[0].hlines(y=10, xmin = 0, xmax = p1end, color='r',linestyle='--', label = '1st phase') 
axs[0].hlines(y=8.333, xmin = p1end, xmax = p2end, color='g',linestyle='--', label = '2nd phase') 
axs[0].hlines(y=5, xmin = p2end, xmax = 22, color='k',linestyle='--', label = '3rd phase') 
#axs[0].axhline(y=10,  linestyle='--',color='r', label = '1st steady state') 
#axs[0].axhline(y=9.09, color='b', linestyle='--', label = '2nd steady state') 
axs[0].set_title("Flow 0",  fontsize=10)

axs[0].vlines(x = p1end, ymin = 8.33, ymax = 10, colors = 'green')
axs[0].vlines(x = p2end, ymin = 5, ymax = 8.33, colors = 'black')


axs[1].plot(x1, y1, marker='.', fillstyle='none', color='y')
axs[1].hlines(y=15, xmin = 0, xmax = p1end, color='r',linestyle='--', label = '1st phase') 
axs[1].hlines(y=8.333, xmin = p1end, xmax = p2end, color='g',linestyle='--', label = '2nd phase') 
axs[1].hlines(y=10, xmin = p2end, xmax = 22, color='k',linestyle='--')#, label = '3rd phase') 
axs[1].set_title("Flow 1",  fontsize=10)

axs[1].vlines(x = p1end, ymin = 8.33, ymax = 15, colors = 'green')
axs[1].vlines(x = p2end, ymin = 8.33, ymax = 10, colors = 'black')



axs[2].plot(x3, y3, marker='.', fillstyle='none', color='y')
axs[2].hlines(y=8.333, xmin = p1end, xmax = p2end, color='g',linestyle='--', label = '2nd phase') 
axs[2].hlines(y=10, xmin = p2end, xmax = 22, color='k',linestyle='--' )#, label = '3rd phase') 
#axs[2].axhline(y=12.5, color='r', linestyle='--', label = '1st steady state')
axs[2].set_title("Flow 2",  fontsize=10)

axs[2].vlines(x = p2end, ymin = 8.33, ymax = 10, colors = 'black')



for ax in axs.flat:
    ax.set (xlabel='Time (us)', ylabel='Flow Injection Rate\n(GB/s)')
    ax.set_xlim(0, 23)
    ax.set_ylim(0, 28)
    #ax.legend()
    ax.label_outer()

fig.set_figheight(1.3)
#fig.set_figwidth(8)
#fig.tight_layout()

# Save the entire figure
fig.savefig("figs/result-3flows-changingrates.png")
